In [ ]:
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate


# read local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

# setup azure openai api
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT")
openai.api_version = "2023-05-15"
openai.api_key  = os.getenv('AZURE_OAI_KEY')

llm_name = "gpt-3.5-turbo"
file_path = os.getenv("FILE_PATH")

deployment_name_gpt = os.getenv("AZURE_OAI_MODEL_GPT_3")
deployment_name_ada = os.getenv("AZURE_OAI_MODEL_ADA")

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")
vector_store_service_name: str = os.getenv("AZURE_SEARCH_SERVICE_NAME")
vector_store_index_name: str = os.getenv("AZURE_SEARCH_INDEX_NAME")

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = vector_store_service_name
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = vector_store_index_name
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = vector_store_password

os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_BASE"] = openai.api_base
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_VERSION"] = openai.api_version

In [ ]:
llm = AzureChatOpenAI(deployment_name=deployment_name_gpt)

In [ ]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=deployment_name_ada, chunk_size=1)
index_name: str = "facts-about-snakes-test1"
acs: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [ ]:
# load documents
loader = PyPDFLoader(file_path)
documents = loader.load()

# split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

In [ ]:
# Add documents to azure cognitive search
acs.add_documents(documents=docs)

In [ ]:
# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
    """Given the following conversation and a follow up question, \
        rephrase the follow up question to be a standalone question.

        Chat History:
        {chat_history}
        Follow Up Input: {question}
        Standalone question:
    """)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=acs.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [37]:
chat_history = []
query = "What are the venomous snakes in Nort Carolina?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: What are the venomous snakes in Nort Carolina?
Answer: The venomous snakes in North Carolina include the Copperhead, Cottonmouth, Timber Rattlesnake, Eastern Diamondback Rattlesnake, Pigmy Rattlesnake, and the Eastern Coral Snake.


In [40]:
chat_history = [(query, result["answer"])]
query = "Which one is the most wide spread?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: Which one is the most wide spread?
Answer: The most common and widespread venomous snake in North Carolina is the Copperhead.


In [41]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
 
    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 
context = []
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard